# Decoding Quantum CSP Codes

In [ ]:
import json
import numpy as np
from tqdm import tqdm
import qecstruct as qc
import qecsim.paulitools as pt

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from matplotlib.ticker import FormatStrFormatter

from mdopt.mps.utils import marginalise, create_custom_product_state
from mdopt.contractor.contractor import mps_mpo_contract
from mdopt.optimiser.utils import (
    SWAP,
    COPY_LEFT,
    XOR_BULK,
    XOR_LEFT,
    XOR_RIGHT,
)
from examples.decoding.decoding import (
    apply_constraints,
    apply_bitflip_bias,
)
from examples.decoding.decoding import (
    decode_css,
    pauli_to_mps,
    css_code_checks,
    css_code_logicals,
    css_code_logicals_sites,
    css_code_constraint_sites,
    generate_pauli_error_string,
)

In [ ]:
import qecstruct

In [ ]:
from qecstruct import LinearCode

In [ ]:
import qecstruct

In [ ]:
from qecstruct import BinaryMatrix

In [ ]:
BinaryMatrix(data["num_qubits"], data["x_stabs"])

In [ ]:
linearcode = LinearCode(BinaryMatrix(data["num_qubits"], data["x_stabs"]))

In [ ]:
x_code = 

In [ ]:
qecstruct.CssCode(x_code=data["x_stabs"], z_code=data["z_stabs"])

In [ ]:
f = open("data-csp/batch_1/codes/qubits_30/code_0.json")
data = json.load(f)

In [ ]:
data["x_stabs"]

In [ ]:
data["x_stabs"]

In [ ]:
NUM_BITS = 60

In [ ]:
state = create_simple_product_state(NUM_BITS, which="+")

In [ ]:
code_constraint_sites = data["x_stabs"]
tensors = [XOR_LEFT, XOR_BULK, SWAP, XOR_RIGHT]
code_constraint_sites